# FaaS Demo

In [ ]:
import requests
import json
import pandas as pd
from flask import Flask, render_template


app = Flask(__name__)

@app.route("/")
def main():
    response = requests.get('http://faas-demo.llnw.net/button')
    response_data = response.json()
    
    return render_template('buttons.html', value = response_data)

@app.route("/button/table")
def aggregate_table():
    response = requests.get('http://faas-demo.llnw.net/button')
    response_data = response.json()
    table_list = []
    for i in response_data:
        if "Table" in i['name']:
            table_list.append(i)
    table = json.dumps(table_list)
    
    df = pd.DataFrame(table_list)
    total_clicks = df['clicks'].sum()
    
    return render_template('table.html', value = table, total_clicks = total_clicks)
            
@app.route("/button/bar")
def aggregate_bar():
    response = requests.get('http://faas-demo.llnw.net/button')
    response_data = response.json()
    array = []
    bar_list = []
    for i in response_data:
        if "Check In" not in i['name'] and "Table" not in i['name'] and "Press" not in i['name']:
            s_resp = requests.get('http://faas-demo.llnw.net/button?stats='+i['mac'])
            s_resp_data = s_resp.json()
            array.append(s_resp_data)
    for i in response_data:
        if "Check In" not in i['name'] and "Table" not in i['name'] and "Press" not in i['name']:
            bar_list.append(i)
    
    bar = json.dumps(bar_list)    
    
    array1 = array[0]['per5min']
    array2 = array[1]['per5min']
    array3 = array[2]['per5min']
    array4 = array[3]['per5min']
    
    ar1 = json.dumps(array1)
    ar2 = json.dumps(array2)
    ar3 = json.dumps(array3)
    ar4 = json.dumps(array4)
    
    df = pd.DataFrame(bar_list)
    total_clicks = df['clicks'].sum()
    
    df2 = pd.DataFrame(array)
    df2['pct_total'] = (df2['clicks'] / df2['clicks'].sum()).round(2)
    df3 = df2.set_index('name')
    
    pct_Beer = df3['pct_total']['Beer']
    Beer_clicks = df3['clicks']['Beer']
    
    pct_Spirits = df3['pct_total']['Spirits']
    Spirits_clicks = df3['clicks']['Spirits']
    
    pct_Soft_Drink = df3['pct_total']['Soft Drink']
    Soft_Drink_clicks = df3['clicks']['Soft Drink']
    
    pct_Wine = df3['pct_total']['Wine']
    Wine_clicks = df3['clicks']['Wine']
  
    s_resp_1 = requests.get('http://faas-demo.llnw.net/button?stats='+i['mac'])
    
    return render_template('bar.html', value = bar, ar1 = ar1, ar2 = ar2, ar3 = ar3, ar4 = ar4, 
                           pct_Beer = pct_Beer, Beer_clicks = Beer_clicks, pct_Spirits = pct_Spirits,
                           Spirits_clicks = Spirits_clicks, pct_Soft_Drink = pct_Soft_Drink, Soft_Drink_clicks = Soft_Drink_clicks,
                           pct_Wine = pct_Wine, Wine_clicks = Wine_clicks, 
                           total_clicks = total_clicks)
    
@app.route("/button/Arrivals")
def button_stats():
    s_resp = requests.get('http://faas-demo.llnw.net/button?stats=b4:7c:9c:73:33:d3')
    s_resp_data = s_resp.json()
    per5min = s_resp_data['per5min']
    detail_stats = json.dumps(per5min)
    return render_template('arrivals.html', value = s_resp_data, 
                           value2 = detail_stats)

@app.route("/button/DoNotPress")
def DoNotPress_stats():
    s_resp = requests.get('http://faas-demo.llnw.net/button?stats=68:37:e9:01:e9:26')
    s_resp_data = s_resp.json()
    per5min = s_resp_data['per5min']
    detail_stats = json.dumps(per5min)
    return render_template('donotpress.html', value = s_resp_data, 
                           value2 = detail_stats)



if __name__ == '__main__':
    app.run(debug=False)